# Case 2: Cargo Ship
The ship MS Leiden is traveling on a route involving five harbors in north-
ern Europe: Rotterdam, Hamburg, Kiel, Aarhus, and Copenhagen. You have
been asked to help the Captain with the loading and unloading plan (for the
destination harbors) that produces

- even and well-balanced solution
- a solution that is easy to unload (so that containers that are earlier un-loaded are not beneath containers that are later unloaded)
- solutions that can integrate as many containers as possible

The following details about the problem are provided:
1. The ship is now docked in Rotterdam. After visiting Rotterdam, the ship
will be traveling to Hamburg, Aarhus, and Copenhagen, in that order. The
containers destined for Rotterdam have been unloaded and the containers
destined for the remaining three harbors have to be loaded on the ship.
2. MS Leiden is a rather small container ship. It has eight bays, three tiers
and four rows. The layout of the ship is shown in the figure below.
3. Containers should be loaded such that a container that has to be unloaded
earlier should be placed in a higher position.
4. Each cell in the above figure is able to hold a forty-foot container. That
is, the ship has room for 8 × 3 × 4 = 96 forty-foot containers.
5. A forty-foot container can be placed on-top of another forty-foot container,
but not on top of an empty cell. We assume that only forty-foot containers
are loaded on the ship. Each container has a destination port.
6. Each container $i$ has a certain weight $w_i$. If a container is much heavier
than another $i$ container $j$, say $w_i − w_j > δ_w$, then it is not allowed to
place $w_i$ on top of $w_j$ .
7. The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [32]:
from desdeo_problem import variable_builder, ScalarObjective, ScalarConstraint, MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder
import numpy as np

# desdeo.probdefs.Problem.__abstractmethods__ = set()
# desdeo docs: https://desdeo.readthedocs.io/en/latest/

![title](cargo1.png)

![title](cargo2.png)

![title](cargo3.png)

In [ ]:
order = ['Rotterdam', 'Hamburg', 'Aarhus', 'Copenhagen']

## Custom dataset

- Load has to be balanced in every part of the journey (Rotterdam -> Hamburg, Hamburg -> Aarhus, Aarhus -> Copenhagen)
- 

In [ ]:
# Different scenarios with different weights (ID, weight)
# The ID is the ID of the cargo and is between 1 and 10
# The weight is between 3750 and 27600 and random
HARBORS = ['Hamburg', 'Aarhus', 'Copenhagen']

# Scenario 1
Hamburg = 1
Aarhus = 2
Copenhagen = 3
min_weight = 1800 # in kg
max_weight = 28000 # in kg
Data = [(1, 5883), (1, 3485), (1, 9985), (1, 17600), 
        (2, 7352), (2, 9230), (2, 4699), (2, 16400), 
        (3, 14500), (3, 5773), (3, 12258)]

# Scenario 2 with different weights (ID, weight)


## Code

Balance score can be calculated as:
The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [29]:
def sum_tiers(configuration):

    only_weights = configuration[:, :, :, 1]
    summed_configuration = np.sum(only_weights, axis=1)

    return summed_configuration

def get_longitudinal_center(summed_configuration):

    # Calculate the longitudinal center of the ship
    # The longitudinal center is the center of mass of the ship
    # The center of mass is the sum of the products of the weights and their positions
    # divided by the sum of the weights

    # Get the weights and the positions of the containers
    weights = np.sum(summed_configuration, axis=1)
    positions = np.arange(1, 9)

    # Calculate the center of mass
    center = np.sum(weights * positions) / np.sum(weights)

    return center

def get_lattitudinal_center(summed_configuration):

    # Calculate the lattitudinal center of the ship
    # The lattitudinal center is the center of mass of the ship
    # The center of mass is the sum of the products of the weights and their positions
    # divided by the sum of the weights

    # Get the weights and the positions of the containers
    weights = np.sum(summed_configuration, axis=0)
    positions = np.arange(1, 5)

    # Calculate the center of mass
    center = np.sum(weights * positions) / np.sum(weights)

    return center

def calculate_centers(configuration):

    summed_configuration = sum_tiers(configuration)

    longitudinal_center = get_longitudinal_center(summed_configuration)
    lattitudinal_center = get_lattitudinal_center(summed_configuration)

    return longitudinal_center, lattitudinal_center

def get_score(configuration):

    longitudinal_center, lattitudinal_center = calculate_centers(configuration)

    long_error, latt_error = abs(longitudinal_center - 4.5), abs(lattitudinal_center - 2.5)

    return long_error + latt_error

def remove_harbor_containers(configuration, destination):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
        destination: integer
            The destination harbor.
    Returns
    -------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    """
    tiers = range(configuration.shape[1])
    for tier in tiers:
        idx_container = np.argwhere(configuration[:, tier, :, 0] == destination)
        for idx in idx_container:
            if idx[1] == 2:
                configuration[idx[0],idx[1],idx[2],:] = (0,0)
            # If the container is not on the top tier, move the container above it down
            else:
                configuration[idx[0],idx[1],idx[2],:] = configuration[idx[0], idx[1]+1, idx[2], :]
                if idx[1] == 1:
                    configuration[idx[0],idx[1]+1,idx[2],:] = (0,0)
                else:
                    configuration[idx[0], idx[1]+1, idx[2], :] = configuration[idx[0], idx[1]+2, idx[2], :]
    return configuration

def calculate_total_stability(configuration):

    stability_score = 0

    for i in range(1, len(HARBORS)+1):

        stability_score += get_score(configuration)
        configuration = remove_harbor_containers(configuration, i)
        
    return stability_score

def get_unloading_time(configuration):
    configuration = np.ones((8, 3, 4, 2))
    only_harbors = configuration[:, :, :, 0]
    # First check
    unloading_time = 0
    for i in range(8):
        for j in range(4):
            stack = only_harbors[i, :, j]
            stack = stack[stack != 0] # remove zeros from stack
            if stack:
                for i in range(len(stack)):
                    # Check how many containers are above the last zero, not including the top one.
                    unloading_time += some_calculation_for_current_harborcontainer

def get_unloading_time(configuration):
    # First check
    unloading_time = 0
    for i in range(8):
        for j in range(4):
            stack = configuration[i, :, j, 0]
            stack = stack[stack != 0] # remove zeros from stack
            if stack:
                for i in range(len(stack)):
                    # Check how many containers are above the last zero, not including the top one.
                    unloading_time += some_calculation_for_current_harborcontainer

    return unloading_time

def get_num_loaded_containers(configuration):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    Returns
    -------
        num_loaded_containers: float
            The total number of loaded containers.
    """
    # First check
    num_loaded_containers = 0
    for i in range(8):
        for j in range(3):
            for k in range(4):
                if configuration[i, j, k, 0] != 0:
                    num_loaded_containers += 1

    return num_loaded_containers


0.2752105622581378

In [ ]:
def ship_matrix(data):

    ship_position = np.zeros((8, 3, 4, 2))

    return ship_position

calculate_total_stability(ship_matrix(Data))

In [ ]:
empty_ship = np.zeros((8, 3, 4, 2))
delta_weight = 500
# Constraints
# 1. A container can only be placed on top of another container or on the floor
# 2. A container can only weigh delta_weight kg more than the container below it
# 
# Objectives
# 1. Minimize the long_error + latt_error (maximize the stability of the ship)
# 2. Minimize the unloading time of the ship


# Args: name, starting value, lower bound, upper bound

# Create variables
var_names = ["c"+str(id) for id in range(len(Data))]

initial_values = [pos for pos in range(8*3*4)]
lower_bounds = [0 for _ in range(8*3*4)]
upper_bounds = [95 for _ in range(8*3*4)]

variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)

# Create objectives
objectives = [ScalarObjective(name="Stability", evaluator=calculate_total_stability, maximize=False),
                ScalarObjective(name="Unloading time", evaluator=get_unloading_time, maximize=False)]

# Create constraints
def overlapping_eval(positions):
    if len(positions) != len(set(positions)):
        return -1
    else:
        return 1
    
def levitating_eval(positions):
    for pos in positions:
        if pos < 32:
            pass
        else:
            pos_below = np.arange(pos-32, -1, -32)
            if not all(x in positions for x in pos_below):
                return -1
    return 1

def weight_eval(positions):
    weights = [Data[pos][1] for pos in positions]
    for i in range(len(weights)-1):
        if weights[i] > weights[i+1] + delta_weight:
            return -1
    return 1

constraints = [ScalarConstraint(name="Overlapping", n_decision_vars=len(Data), n_objective_funs=0, evaluator=overlapping_eval),
               ScalarConstraint(name="Levitating", n_decision_vars=len(Data), n_objective_funs=0, evaluator=levitating_eval),
               ScalarConstraint(name="Weight", n_decision_vars=len(Data), n_objective_funs=0, evaluator=weight_eval)]

problem = MOProblem(objectives, variables, constraints)

h = np.random.randint(1, 3, 100)
w = np.random.randint(2000, 30000, 100)
Data = np.zip(h,w)

res = problem.evaluate(Data)